In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers


In [2]:
import torch
import pandas as pd
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel

In [3]:
# 加载News.xlsx文件
news_df = pd.read_excel('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/News.xlsx')


In [4]:
news_df.info

<bound method DataFrame.info of           NewsID                                    Title  \
0              1                       建设银行原董事长张恩照一审被判15年   
1              2                             农行信用卡中心搬到上海滩   
2              3                        外运发展：价值型蓝筹股补涨要求强烈   
3              4                        胜利股份：稳步走强形成标准上升通道   
4              5               [港股快讯]恒指收市报18960点 成交467亿港元   
...          ...                                      ...   
1037030  1037031    亿华通：公司电解槽相关产品目前还处于产品的研发及测试阶段 尚未实现批量销售   
1037031  1037032                             依米康：接受中泰证券调研   
1037032  1037033         天风证券给予中核科技买入评级 核电行业景气上行 公司有望乘风而起   
1037033  1037034  海特生物：公司在抗癌药CPT获批后 会考虑适时开展CPT在海外的临床并谋求上市   
1037034  1037035                      恩捷股份：股东合益投资部分股份补充质押   

                                               NewsContent NewsSource  
0        　　本报记者 田雨 李京华    　　中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...      中国证券报  
1        　　中国农业银行信用卡中心由北京搬到上海了！  　　农行行长杨明生日前在信用卡中心揭牌仪式上...       人民日报  
2        　　在新基金快速发行

In [ ]:
news_df.head(1000).to_csv('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/new_example.csv', index=False)

In [ ]:
news_df_example = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/new_example.csv')

In [ ]:
column_names = news_df_example.columns.tolist()
print(column_names)
news_df_example.loc[:1]

In [ ]:
news_df_test = news_df_example[:10]
news_df_test

In [ ]:
# 加载A_share_list.json文件
with open('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/A_share_list.json', 'r') as f:
    a_share_list = json.load(f)

# 将A_share_list转换为DataFrame
a_share_df = pd.DataFrame(a_share_list)

In [ ]:
column_names = a_share_df.columns.tolist()
print(column_names)

# bert-base-chinese

In [ ]:
# 初始化BERT tokenizer和model
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
model = AutoModel.from_pretrained('bert-base-chinese')

In [ ]:

# 对公司简名进行向量化
company_name = []
for company in a_share_df['name']:
    encoded_input = tokenizer(company, padding=True, truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    company_vector = model_output.pooler_output.numpy()[0]
    company_name.append(company_vector)

company_name = np.stack(company_name) #列表转换为NumPy数组


In [ ]:
# 对公司全名进行向量化
company_fullname = []
for company in a_share_df['fullname']:
    company = company.replace("股份有限公司", "").replace("集团", "")
    encoded_input = tokenizer(company, padding=True, truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    company_vector = model_output.pooler_output.numpy()[0]
    company_fullname.append(company_vector)

company_fullname = np.stack(company_fullname) #列表转换为NumPy数组

试错，后删

In [ ]:
import re
import jieba
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag

# 下载停用词表
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
# 加载停用词表
stopwords = set(stopwords.words('chinese'))  # 使用NLTK内置的中文停用词表

# 需要去除的特定词汇列表
specific_words = ["股份有限公司", "集团"]

def preprocess(text):
    # 去除特殊字符和标点符号
    text = re.sub(r"[^\w\s]", "", text)

    # 去除多余的空格和换行符
    text = re.sub(r"\s+", " ", text).strip()


    # 分词
    words = jieba.lcut(text)

    # 去除停用词
    words = [word for word in words if word not in stopwords]

    # 词性标注
    tagged_words = pos_tag(words)

    # 过滤名词
    filtered_nouns = [word for word, pos in tagged_words if pos.startswith('N') and word not in specific_words]
    filtered_nouns = [noun for noun in filtered_nouns if noun.strip() != '']

    return filtered_nouns

正确的预处理

In [ ]:
!pip install thulac

In [ ]:
import re
import thulac
import nltk
from nltk.corpus import stopwords

# 下载停用词表
nltk.download('stopwords')
# 加载停用词表
stopwords = set(stopwords.words('chinese'))  # 使用NLTK内置的中文停用词表

# 需要去除的特定词汇列表
specific_words = ["股份","有限公司", "集团"]  # 添加你想去除的特定词汇


In [ ]:
def preprocess(text):
    # 初始化THULAC实例

    thu = thulac.thulac()

    # 分词和词性标注
    result = thu.cut(text, text=True)



     # 过滤名词
    nouns = []

    for word_pos in result.split():
        word, pos = word_pos.split('_')
        if pos == 'n' and word not in specific_words and word not in stopwords:
            nouns.append(word)

    return nouns

# 测试
#news_content = "本报记者 田雨 李京华 　　中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣判，北京市第一中级人民法院依法以受贿罪判处张恩照有期徒刑１５年。 　　法院经开庭审理查明，２０００年至２００４年期间，被告人张恩照利用其担任原中国建设银行副行长、行长，中国建设银行股份有限公司董事长的职务便利，为他人牟取利益，多次非法收受他人给予的款物共计人民币４００余万元。案发后，赃款、赃物已全部退缴。 　　法院认为，被告人张恩照身为国家工作人员，利用职务上的便利，为他人谋取利益，非法收受他人财物，其行为已构成受贿罪，受贿数额特别巨大。鉴于张恩照因其他违纪问题被审查后，主动交代了有关部门不掌握的本案受贿事实，属于自首，且赃款、赃物已全部退缴，对张恩照依法可从轻处罚。法院遂依法以受贿罪判处张恩照有期徒刑１５年。"
#filtered_nouns = preprocess(news_content)
#print("".join(filtered_nouns))

In [ ]:

# 定义阈值，阈值和新闻文本中的噪音长度有关
threshold = 0.8
# 存储有效新闻的列表
valid_news = []

# 遍历新闻条目
for index, news_row in news_df_test.iterrows():
    # 提取新闻
    newsID = news_row['NewsID']
    news_title = news_row['Title'] #
    news_content = news_row['NewsContent']

    filtered_nouns = preprocess(news_content)

    content = news_title+"".join(filtered_nouns)



    # 将新闻向量化
    encoded_input = tokenizer(content, padding=True, truncation=True, max_length=128, return_tensors='pt')

    name = ['建设银行']
    answer = tokenizer(name, padding=True, truncation=True, max_length=128, return_tensors='pt')
    #tokens = tokenizer.tokenize(content)
    #token_count = len(tokens)
    #print("Token count:", token_count)

    with torch.no_grad():
        model_output = model(**encoded_input)
    news_vector = model_output.pooler_output.numpy()[0]
    #print(news_vector.shape) (768,)
    with torch.no_grad():
        model_output = model(**answer)
    answer_vector = model_output.pooler_output.numpy()[0]
    #print(news_vector.shape) (768,)
    # 计算新闻向量与公司向量之间的相似度
    #company_vectors.shape (4654, 768)
    similarities = cosine_similarity(news_vector.reshape(1, -1), company_name)
    #print(similarities.shape) (1, 4654)


    similarity_a = cosine_similarity(news_vector.reshape(1, -1), answer_vector.reshape(1, -1))
    print("正确答案的相似度：",similarity_a)
    # 检查是否有相似度超过阈值的公司
    # 获取所有相似度大于阈值的公司索引
    max_similarity_index = np.argmax(similarities)
    similar_indices = np.where(similarities[0, :] > threshold)[0]
    print(similar_indices.size)





    if similar_indices.size > 0:

      max_similarity = similarities[:,max_similarity_index]
      min_similarity = np.min(similarities[0, similar_indices])
      print("最大相似度：", max_similarity)
      print("最小相似度：", min_similarity)
      #print(similarities.shape)           # 打印相似度数组的形状(1, 4654)
      #print(similar_indices.shape)        # 打印索引数组的形状(12,)

      result_df = pd.DataFrame({
          'Similarity': similarities[0,similar_indices],
          'Company Name': a_share_df.loc[similar_indices, 'name']
      })

      print(result_df)

      # 获取所有相似度大于阈值的公司名称
      Explicit_companies = a_share_df.loc[similar_indices, 'name'].tolist()
      #Explicit_companies = a_share_df.loc[max_similarity_index, 'name']
      valid_news.append((newsID,content,Explicit_companies))


# 将有效新闻转换为DataFrame
valid_news_df = pd.DataFrame(valid_news)

In [ ]:
data = {
    'Index': [0],
    'NewsID': [1],
    'Title': ['建设银行'],
    'NewsContent': ['建设银行原董事长张恩照一审被判15年记者田雨银行董事长受贿案一审受贿罪法院被告人银行行长行长银行董事长职务利益款物人民币赃款赃物法院被告人国家人员职务利益财物受贿罪数额部门事实赃款赃物法院受贿罪'],
    'NewsSource': ['中国证券报']
}

news_df_test = pd.DataFrame(data)

In [ ]:
valid_news_df

# BERT-wwm

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

In [ ]:
# 加载tokenizer和模型
tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-bert-wwm')
model = AutoModel.from_pretrained('hfl/chinese-bert-wwm')

# 对公司简名进行向量化
company_name = []
for company in a_share_df['name']:
  # 使用tokenizer对文本进行分词和编码
  tokens = tokenizer.encode_plus(company,add_special_tokens=True,padding='max_length',truncation=True,max_length=128,return_tensors='pt')
  # 获取模型的输入张量
  input_ids = tokens['input_ids']
  attention_mask = tokens['attention_mask']

  # 使用模型进行向量化
  with torch.no_grad():
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)

  # 获取文本的向量表示
  company_vector = outputs.last_hidden_state.mean(dim=1).squeeze()
  company_name = [company_vector.detach().numpy() for vector in company_vector]
  company_name = np.stack(company_name)




In [ ]:
# 定义阈值，阈值和新闻文本中的噪音长度有关
threshold = 0.8
# 存储有效新闻的列表
valid_news = []

# 遍历新闻条目
for index, news_row in news_df_test.iterrows():
    # 提取新闻
    newsID = news_row['NewsID']
    news_title = news_row['Title'] #
    news_content = news_row['NewsContent']

    filtered_nouns = preprocess(news_content)

    content = news_title+"".join(filtered_nouns)



    # 将新闻向量化
    tokens = tokenizer.encode_plus(content,add_special_tokens=True,padding='max_length',truncation=True,max_length=128,return_tensors='pt')
    encoded_input = tokenizer(content, padding=True, truncation=True, max_length=128, return_tensors='pt')

    name = ['建设银行']
    answer = tokenizer(name, padding=True, truncation=True, max_length=128, return_tensors='pt')
    #tokens = tokenizer.tokenize(content)
    #token_count = len(tokens)
    #print("Token count:", token_count)

    with torch.no_grad():
        model_output = model(**encoded_input)
    news_vector = model_output.pooler_output.numpy()[0]
    #print(news_vector.shape) (768,)
    with torch.no_grad():
        model_output = model(**answer)
    answer_vector = model_output.pooler_output.numpy()[0]
    #print(news_vector.shape) (768,)
    # 计算新闻向量与公司向量之间的相似度
    #company_vectors.shape (4654, 768)
    similarities = cosine_similarity(news_vector.reshape(1, -1), company_name)
    #print(similarities.shape) (1, 4654)


    similarity_a = cosine_similarity(news_vector.reshape(1, -1), answer_vector.reshape(1, -1))
    print("正确答案的相似度：",similarity_a)
    # 检查是否有相似度超过阈值的公司
    # 获取所有相似度大于阈值的公司索引
    max_similarity_index = np.argmax(similarities)
    similar_indices = np.where(similarities[0, :] > threshold)[0]
    print(similar_indices.size)





    if similar_indices.size > 0:

      max_similarity = similarities[:,max_similarity_index]
      min_similarity = np.min(similarities[0, similar_indices])
      print("最大相似度：", max_similarity)
      print("最小相似度：", min_similarity)
      #print(similarities.shape)           # 打印相似度数组的形状(1, 4654)
      #print(similar_indices.shape)        # 打印索引数组的形状(12,)

      result_df = pd.DataFrame({
          'Similarity': similarities[0,similar_indices],
          'Company Name': a_share_df.loc[similar_indices, 'name']
      })

      print(result_df)

      # 获取所有相似度大于阈值的公司名称
      Explicit_companies = a_share_df.loc[similar_indices, 'name'].tolist()
      #Explicit_companies = a_share_df.loc[max_similarity_index, 'name']
      valid_news.append((newsID,content,Explicit_companies))


# 将有效新闻转换为DataFrame
valid_news_df = pd.DataFrame(valid_news)

# 关键词匹配

In [ ]:
import pandas as pd
import pickle
import math
# 公司名称列表，假设为一个包含公司名称和公司全名的列表
company_name = a_share_df['name']
company_fullname = a_share_df['fullname']

# 定义检查点文件路径
checkpoint_file = '/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/checkpoint/filtered_news_checkpoint.pkl'
# 定义保存检查点的频率
save_checkpoint_freq = 10000

# 记录已处理的新闻数量
processed_news_count = 0

# 检查是否存在检查点文件
try:
    # 尝试加载检查点文件
    with open(checkpoint_file, 'rb') as f:
        filtered_df = pickle.load(f)
except FileNotFoundError:
    # 创建新的数据框用于存储包含公司名的新闻
    filtered_df = pd.DataFrame(columns=['NewsID', 'NewsContent', 'Explicit_Company'])

# 遍历每条新闻文本
for index, row in news_df.iterrows():
    newsID = row['NewsID']
    news_title = row['Title']
    news_content = row['NewsContent']

    # 跳过包含缺失值的新闻内容
    if isinstance(news_content, float) and math.isnan(news_content):
        continue

    # 在新闻文本中寻找匹配公司名称的部分
    explicit_company = []

    # 在公司名称列表中进行关键词匹配
    for name, fullname in zip(company_name, company_fullname):
        if name in news_content or fullname in news_content:
            explicit_company.append(name)

    if explicit_company:
        # 找到匹配的公司名称，将新闻添加到新的数据框中
        filtered_df = pd.concat([filtered_df, pd.DataFrame({'NewsID': [newsID], 'NewsContent': [news_content], 'Explicit_Company': [explicit_company]})], ignore_index=True)

    # 增加已处理的新闻数量
    processed_news_count += 1

    # 当达到保存检查点的频率时，保存检查点
    if processed_news_count % save_checkpoint_freq == 0:
        with open(checkpoint_file, 'wb') as f:
            pickle.dump(filtered_df, f)

# 最后一次保存检查点
with open(checkpoint_file, 'wb') as f:
    pickle.dump(filtered_df, f)

# 打印筛选后的新闻数据框
#print(filtered_df)

# 保存有效新闻到新的文档
filtered_df.to_excel('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/Q1/filtered_news_1.xlsx', index=False)

In [ ]:
import pandas as pd
# 公司名称列表，假设为一个包含公司名称和公司全名的列表
company_name = a_share_df['name']
company_fullname = a_share_df['fullname']

# 创建新的数据框用于存储包含公司名的新闻
filtered_df = pd.DataFrame(columns=['NewsID','NewsContent','Explicit_Company'])

# 遍历每条新闻文本
for index, row in news_df_example.iterrows():

    newsID = row['NewsID']
    news_title = row['Title']
    news_content = row['NewsContent']

    # 在新闻文本中寻找匹配公司名称的部分
    explicit_company = []

    # 在公司名称列表中进行关键词匹配
    for name, fullname in zip(company_name, company_fullname):
        if name in news_content or fullname in news_content:
            explicit_company.append(name)

    if explicit_company:
        # 找到匹配的公司名称，将新闻添加到新的数据框中
        filtered_df = pd.concat([filtered_df, pd.DataFrame({'NewsID': [newsID], 'NewsContent': [news_content], 'Explicit_Company': [explicit_company]})], ignore_index=True)


# 打印筛选后的新闻数据框
#print(filtered_df)
# 保存有效新闻到新的文档
filtered_df.to_excel('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/Q1/filtered_news_1.xlsx', index=False)